In [1]:
from backpack_predictor import prepare_data, target_encoding
from backpack_predictor.features import target, baseline_features, feature_list, cat_cols

%load_ext autoreload
%autoreload 2

from datetime import datetime
import time
import numpy as np
import pandas as pd
from scipy.stats import skew, chisquare, kruskal, ks_2samp, chi2_contingency

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import root_mean_squared_error

import xgboost as xgb
import lightgbm as lgb

from autogluon.tabular import TabularPredictor

import warnings
warnings.filterwarnings('ignore')

test_df = pd.read_csv(r'..//data//test.csv')
train_df = pd.read_csv(r'..//data//train.csv')
train_extra_df = pd.read_csv(r'..//data//training_extra.csv')
train_df = pd.concat([train_df, train_extra_df], ignore_index=True)


# Apply function to train and test datasets
train_df = prepare_data(train_df, is_train=True)
test_df = prepare_data(test_df, is_train=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
train_df.columns

Index(['brand', 'material', 'size', 'compartments', 'laptop_compartment',
       'is_waterproof', 'style', 'color', 'weight_capacity', 'price'],
      dtype='object')

In [6]:
feature_list = [
    'compartments', 'weight_capacity_te', 'laptop_compartment', 'is_waterproof',
]+ [target]

feature_list = ['brand', 'material', 'size', 'compartments', 'laptop_compartment',
       'is_waterproof', 'style', 'color', 'weight_capacity', 'price']

In [7]:
predictor = TabularPredictor(
    label=target, 
    problem_type='regression', 
    eval_metric='root_mean_squared_error'
).fit(
    train_df[feature_list], 
    presets='best',
    time_limit=60 * 60 * 7
)

No path specified. Models will be saved in: "AutogluonModels/ag-20250214_033422"


Preset alias specified: 'best' maps to 'best_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.10
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.3.0: Thu Jan  2 20:24:16 PST 2025; root:xnu-11215.81.4~3/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       28.61 GB / 64.00 GB (44.7%)
Disk Space Avail:   457.75 GB / 926.35 GB (49.4%)
Presets specified: ['best']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of A

(_ray_fit pid=46970) [1000]	valid_set's rmse: 38.8629


(_dystack pid=46504) 	-38.8822	 = Validation score   (-root_mean_squared_error)
(_dystack pid=46504) 	148.29s	 = Training   runtime
(_dystack pid=46504) 	154.16s	 = Validation runtime
(_dystack pid=46504) Fitting model: LightGBM_BAG_L1 ... Training model for up to 3732.70s of the 5832.21s of remaining time.
(_dystack pid=46504) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=5.16%)
(_dystack pid=46504) 	-38.8704	 = Validation score   (-root_mean_squared_error)
(_dystack pid=46504) 	71.91s	 = Training   runtime
(_dystack pid=46504) 	70.57s	 = Validation runtime
(_dystack pid=46504) Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 3647.32s of the 5746.82s of remaining time.
(_dystack pid=46504) 	-38.8772	 = Validation score   (-root_mean_squared_error)
(_dystack pid=46504) 	592.63s	 = Training   runtime
(_dystack pid=46504) 	63.82s	 = Validation runtime
(_dystack pid=46504) Fitting model: CatB

(_ray_fit pid=48529) [1000]	valid_set's rmse: 38.8586 [repeated 8x across cluster]
(_ray_fit pid=48529) [2000]	valid_set's rmse: 38.8589 [repeated 5x across cluster]


(_dystack pid=46504) 	-38.8602	 = Validation score   (-root_mean_squared_error)
(_dystack pid=46504) 	227.4s	 = Training   runtime
(_dystack pid=46504) 	102.51s	 = Validation runtime
(_dystack pid=46504) Fitting model: LightGBM_BAG_L2 ... Training model for up to 1844.60s of the 1844.34s of remaining time.
(_dystack pid=46504) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=7.79%)
(_dystack pid=46504) 	-38.862	 = Validation score   (-root_mean_squared_error)
(_dystack pid=46504) 	35.19s	 = Training   runtime
(_dystack pid=46504) 	18.17s	 = Validation runtime
(_dystack pid=46504) Fitting model: RandomForestMSE_BAG_L2 ... Training model for up to 1802.56s of the 1802.30s of remaining time.
(_dystack pid=46504) 	Warning: Reducing model 'n_estimators' from 300 -> 53 due to low time. Expected time usage reduced from 10058.2s -> 1800.2s...
(_dystack pid=46504) 	-38.9249	 = Validation score   (-root_mean_squared_err

In [8]:
predictor = TabularPredictor.load(
    "/Users/jordanbarker/Documents/Kaggle/backpack-prediction-challenge/notebooks/AutogluonModels/ag-20250214_033422"
)
preds = predictor.predict(test_df)

In [9]:
model_str = 'ag-20250214_03342'

In [10]:
submit_path = f'..//submissions//' + model_str + ".csv"
print("Saving to:", submit_path)

submit_df = test_df[['id']].copy()
submit_df['Price'] = preds
submit_df.to_csv(submit_path, index=False)
submit_df.head(5)

Saving to: ..//submissions//ag-20250214_03342.csv


,id,Price
0,300000,81.542183
1,300001,82.188515
2,300002,83.514381
3,300003,81.122070
4,300004,79.769196
